+ [x] import swedbank's homepage with the interest rates, "listränta" (https://www.swedbank.se/privat/boende-och-bolan/bolanerantor.html)
+ [x] export the actual listränta
  + ~~use some grep function to locate '3 mån' and save the next value. But also, make sure it's the interest!~~
  + [x] extract: date, 3 month and years; 1, 2, 3, 4, 5, 6, 7, 8, 9, 10. Ok to hard code index
  + [x] save into a csv, but only if the numbers have updated
+ make script run itself twice a day, github?
+ plot it


>use bensinpris.ipynb as inspiration

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

url = "https://www.swedbank.se/privat/boende-och-bolan/bolanerantor.html"

# Extract the html page
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

# locate the 'td' element
interests = soup.find_all('td', attrs={'class':'JustifyLeft'})

# extract interest rates
# use indexes, 29:49
values = []
for i in range(29, 49 + 1, 2): # +1 so 49 is included
    val = interests[i].text

    # remove white space & extra characters
    val = val.replace("%", "")
    val = val.replace("\n", "")
    val = val.replace(",", ".")
    val = val.strip()
    val = float(val)
    
    values.append(val)

# date + values
date = [str(datetime.date.today())]
dat = pd.DataFrame([date + values])

# column names
colnames = ['date', '3m', '1y', '2y', '3y', '4y', '5y', '6y', '7y', '8y', '9y', '10y']
dat.columns = colnames

In [5]:
import os
import numpy as np

def save_table(filename, dat):
    
    if not os.path.isfile(filename):
        
        # First time file is generated, include the header
        print(f"file {filename} is not found, generating one...")
        dat.to_csv(filename, header = True, index = False)
    
    else:

        # Extract the last row from database (exclude the date), make into array
        df = pd.read_csv(filename)
        last_row = np.array(df.iloc[-1, 1:])
        data = np.array(dat.iloc[:, 1:])

        test = last_row == data

        # if value is different, append to the csv
        if not test.all():
            print(f"updating file {filename}...")
            dat.to_csv(filename, header = False, index = False, mode = 'a')
        else:
            print(f"\nnothing to update in file {filename}\n\n")
            print(f"Today's interests rates are:\n{dat}")

In [121]:
filename = 'swedbank.csv'
save_table(filename, dat)

nothing to update in file swedbank.csv


---

under construction below...

In [8]:
print(f"Today's interests rates are\n", f"{dat}")

Today's interests rates are
          date    3m    1y    2y    3y    4y    5y    6y    7y    8y    9y  \
0  2024-08-27  5.49  4.24  3.84  3.74  3.74  3.74  3.74  3.74  3.74  3.74   

    10y  
0  3.74  


In [170]:
# Check if the date already exist in the csv, not to add unnecessary rows
def update_table():
    d = pd.read_csv('data/bensinpris.csv', sep='\t')
    
    # If no previous file exists save with header
    #if ():
    #    df.to_csv('data/bensinpris.csv', sep='\t', header=True, mode='a', index=False)
    #    print('Table created')
    
    if (all(d.date != str(today))):
        print('Table updating...')
        df.to_csv('data/bensinpris.csv', sep='\t', header=False, mode='a', index=False)
    
    if any(d.date == str(today)):
        d1 = d.loc[d['date'] == str(today)]
        
        if (str(d1.price) == str(df.price)):
            print('Table already updated')
        
        if (str(d1.price) != str(df.price)):
            print('Table updating, replacing todays date')
            d = d[:-1] # Remove the last row, should be the latest date
            d = d.append(df, ignore_index=True)
            
            # Replace the previous csv
            d.to_csv('data/bensinpris.csv', sep='\t', header=True, mode='w', index=False)

update_table()

,date,3m,1y,2y,3y,4y,5y,6y,7y,8y,9y,10y
0,2024-08-26,5.49,4.24,3.84,3.74,3.74,3.74,3.74,3.74,3.74,3.74,3.74


# B

# 

# 